<a href="https://colab.research.google.com/github/joaopdss/segmentation-system/blob/main/semantic_segmentation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [ ]:
!7za x /content/drive/MyDrive/Datasets/mapillary.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Datasets/                                          1 file, 4675505893 bytes (4459 MiB)

Extracting archive: /content/drive/MyDrive/Datasets/mapillary.zip
  0% 256 Open               0% 512 Open               1% 1024 Open                1% 1280 Open                2% 1792 Open                3% 2048 Open                3% 2560 Open                4% 2816 Open                4% 3072 Open              

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Conv2DTranspose, concatenate, Input
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import os
print(len(os.listdir("/content/mapillary/training/images")))
print(len(os.listdir("/content/mapillary/validation/images")))
print(len(os.listdir("/content/mapillary/training/v2.0/labels")))
print(len(os.listdir("/content/mapillary/validation/v2.0/labels")))

17999
2000
17999
2000


In [ ]:
import gc

def read_process(train_image_path, train_mask_path):
  train_set, val_set = None, None

  for dir in ["training", "validation"]:
    path_img = train_image_path.format(dir)
    path_mask = train_image_path.format(dir)

    images = []
    masks = []

    for image in os.listdir(path_img):
      img = tf.io.read_file(f"{path_img}/{image}")
      img = tf.io.decode_png(img)
      with tf.device("CPU"):
        img = tf.image.convert_image_dtype(img, tf.float16)
      # img = tf.cast(img, tf.float32)
      images.append(img)

    print("foi")
    for mask in os.listdir(path_mask):
      mask = tf.io.read_file(f"{path_mask}/{image}")
      mask = tf.io.decode_png(mask)
      with tf.device("CPU"):
        mask = tf.image.convert_image_dtype(mask, tf.float16)
      # mask = tf.cast(mask, tf.float32)
      masks.append(mask)

    if dir == "training":
      with tf.device("CPU"):
        train_set = tf.data.Dataset.from_tensor_slices((images, masks)).prefetch(tf.data.AUTOTUNE)
    else:
      val_set = tf.data.Dataset.from_tensor_slices((images, masks)).prefetch(tf.data.AUTOTUNE)

  return train_set, val_set

In [ ]:
train_set, val_set = read_process("/content/mapillary/{0}/images", "/content/mapillary/{0}/v2.0/labels")

In [ ]:
train_set = train_set.batch(32)
val_set = val_set.batch(32)

In [ ]:
for i in train_set.batch(32).take(1):
  fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(5,5))

  axes[0].imshow(i[0][0] * 255)
  axes[1].imshow(i[0][1] * 255)
  break

## Implementing U-Net architecture

In [ ]:
def encoder_block(inputs, n_filters=64, dropout_prob=0.5, max_pooling=True):
  conv = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal')(inputs)
  conv = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal')(conv)
  conv = BatchNormalization()(conv, training=False)
  conv = Dropout(dropout_prob)(conv)

  if max_pooling:
    next_layer = MaxPooling2D(pool_size=(2,2))(conv)
  else:
    next_layer = conv

  skip_connection = conv
  return next_layer, skip_connection


def decoder_block(prev_layer_input, skip_layer_input, n_filters=64):
  up = Conv2DTranspose(n_filters,
                       (3,3),
                       strides=(2,2),
                       padding='same')(prev_layer_input)
  merge = concatenate([up, skip_layer_input], axis=3)

  conv = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal')(merge)
  conv = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal')(conv)
  return conv

In [ ]:
def unet_compiled(input_size=(512, 256, 3), n_filters=64, n_classes=3):
  inputs = Input(input_size)

  # Encoder
  cblock1 = encoder(inputs, n_filters,dropout_prob=0, max_pooling=True)
  cblock2 = encoder(cblock1[0],n_filters*2,dropout_prob=0, max_pooling=True)
  cblock3 = encoder(cblock2[0], n_filters*4,dropout_prob=0, max_pooling=True)
  cblock4 = encoder(cblock3[0], n_filters*8,dropout_prob=0.5, max_pooling=True)
  cblock5 = encoder(cblock4[0], n_filters*16, dropout_prob=0.3, max_pooling=False)

  # Decoder
  ublock6 = decoder(cblock5[0], cblock4[1],  n_filters * 8)
  ublock7 = decoder(ublock6, cblock3[1],  n_filters * 4)
  ublock8 = decoder(ublock7, cblock2[1],  n_filters * 2)
  ublock9 = decoder(ublock8, cblock1[1],  n_filters)

  conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

  conv10 = Conv2D(n_classes, 1, padding='same')(conv9)

  # Define the model
  model = tf.keras.Model(inputs=inputs, outputs=conv10)

  return model